In [1]:
from surprise.dataset import DatasetAutoFolds
from surprise import Reader
import pandas as pd

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
ratings.to_csv('../data/ml-latest-small/ratings_noh.csv', index=False, header=False)
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='../data/ml-latest-small/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [2]:
from surprise import SVD

algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [3]:
movies = pd.read_csv('../data/ml-latest-small/movies.csv')
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds == 42].count() == 0:
    print('userId 9는 movieId 42의 평점 없음')
print(movies[movies['movieId'] == 42])

userId 9는 movieId 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [4]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [5]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print(f'평점 매긴 영화수: {len(seen_movies)}, 추천대상 영화수: {len(unseen_movies)}, 전체 영화수: {len(total_movies)}')
    return unseen_movies

In [6]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [7]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('#### Top 10 추천 영화 리스트 ####')

for top_movie in top_movie_preds:
    print(f'{top_movie[1]} : {top_movie[2]}')

평점 매긴 영화수: 46, 추천대상 영화수: 9696, 전체 영화수: 9742
#### Top 10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
